In [1]:
from dotenv import load_dotenv
import os

# .env에서 API 키 로드
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# PDF 로드
loader = PyPDFLoader("data/5._Linear_Regression.pdf")
documents = loader.load()

# 텍스트를 작은 조각으로 분할 (RAG에 적합하도록)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# OpenAI 임베딩 설정
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# 벡터 저장소 생성
vectorstore = Chroma.from_documents(texts, embeddings)

/var/folders/fn/7bjmjvmn2d30b47j1yfz_5jm0000gn/T/ipykernel_57810/4074028122.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)


In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key)

# 벡터 저장소 (이미 생성된 texts 사용)
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
vectorstore = Chroma.from_documents(texts, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 1. 요약 프롬프트
prompt1 = PromptTemplate(
    input_variables=["context"],
    template="""
    다음 강의 자료를 기반으로 주요 내용을 상세하게 요약해줘. 
    - "이것은 abc에 대한 내용입니다" 같은 추상적 설명은 피하고, 자료의 실제 내용을 기반으로 구체적으로 작성해줘.
    - 각 주요 항목(예: 섹션, 주제)을 구분하고, 항목별로 상세한 설명(최소 2~3문장)을 포함해줘.
    - 전체 요약은 500~700자 정도로, 한 장 분량으로 정리해줘.
    자료: {context}
    """
)

# 2. 퀴즈 프롬프트
prompt2 = PromptTemplate(
    input_variables=["context"],
    template="""
    다음 강의 자료를 기반으로 고급 수준의 객관식 퀴즈 5개와 주관식 퀴즈 2개를 만들어줘. 
    퀴즈는 단순 암기나 정의 확인이 아니라, 자료의 개념을 깊이 이해하고 응용력을 요구해야 해.
    - 객관식: 각 질문은 4개의 선택지를 포함하며, 선택지는 혼동을 주지 않고 사고를 자극해야 함. 형식:
      Q1. 질문
      a) 선택지1  b) 선택지2  c) 선택지3  d) 선택지4
      정답: a
    - 주관식: 분석, 비교, 또는 문제 해결을 요구하며, 정답은 간결한 문장으로.
    자료: {context}
    """
)

# 3. 레포트 프롬프트
prompt3 = PromptTemplate(
    input_variables=["summary", "quizzes"],
    template="""
    다음 정보를 기반으로 강의 자료 레포트를 작성해줘. 
    레포트는 아래 구조를 따라야 해:
    ---
    # 강의 자료 레포트
    ## 1. 강의 내용 요약
    {summary}

    ## 2. 강의 내용 퀴즈
    {quizzes}
    ---
    """
)

# 체인 정의
chain1 = prompt1 | llm | StrOutputParser()  # 요약 생성
chain2 = prompt2 | llm | StrOutputParser()  # 퀴즈 생성
chain3 = prompt3 | llm | StrOutputParser()  # 레포트 작성

# RAG 검색을 위한 컨텍스트 생성 함수
def get_context(input):
    docs = retriever.get_relevant_documents(input["query"])
    return {"context": "\n".join([doc.page_content for doc in docs])}

# 병렬 실행 설정
parallel_chain = RunnableParallel(
    {
        "summary": RunnablePassthrough.assign(**{"context": get_context}) | chain1,
        "quizzes": RunnablePassthrough.assign(**{"context": get_context}) | chain2
    }
)

# 전체 파이프라인 구성
overall_chain = (
    {"query": RunnablePassthrough()}  # 입력 query 전달
    | parallel_chain                  # 요약과 퀴즈 병렬 실행
    | chain3                          # 레포트 생성
)

# 실행
result = overall_chain.invoke("강의 자료 정리와 퀴즈 생성을 위한 레포트를 작성해줘")
print(f"레포트:\n{result}")

/var/folders/fn/7bjmjvmn2d30b47j1yfz_5jm0000gn/T/ipykernel_57810/3691347973.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key)
/var/folders/fn/7bjmjvmn2d30b47j1yfz_5jm0000gn/T/ipykernel_57810/3691347973.py:66: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(input["query"])


레포트:
# 강의 자료 레포트

## 1. 강의 내용 요약

### 1. 선형 회귀 모델(Linear Regression)
선형 회귀 모델은 독립 변수(X)와 종속 변수(Y) 간의 관계를 선형적으로 모델링하는 통계적 기법으로, 데이터 점들을 기반으로 최적의 직선을 찾아 예측을 수행한다. 이 모델은 회귀 계수(β)를 통해 각 독립 변수의 중요도를 평가할 수 있으며, 예측된 Y값은 주어진 X값에 의해 설명될 수 있다. 선형 회귀는 예측 및 추세 분석에 널리 사용되며, 간단하고 해석하기 쉬운 특성으로 인해 많이 활용된다.

### 2. 데이터 분할(Training and Test Sets)
모델의 일반화 능력을 평가하기 위해 데이터는 훈련 데이터와 테스트 데이터로 분할된다. 훈련 데이터는 모델을 학습하는 데 사용되며, 테스트 데이터는 학습된 모델의 성능을 검증하는 데 활용된다. 이 과정은 과적합(overfitting)을 방지하고 모델의 예측력을 평가하는 데 필수적이다. 일반적으로 데이터의 70-80%는 훈련에, 나머지 20-30%는 테스트에 할당되는 것이 일반적이다.

### 3. 모델 구축 및 테스트
훈련 데이터를 사용하여 모델을 구축한 후, 이를 통해 예측값을 생성한다. 예를 들어, 훈련 데이터에서 얻은 입력(X_tr)을 기반으로 종속 변수(Y_tr)를 예측하는 모델(f)을 구성하고, 이후 테스트 데이터(X_te)를 이용해 모델의 예측 성능을 평가한다. 이 과정을 통해 모델이 실제 데이터에 대한 반응을 얼마나 잘 검증하는지를 확인할 수 있다.

이 자료는 선형 회귀 모델의 기본 개념과 데이터 분할 방법론, 그리고 모델 구축 및 테스트 과정을 상세히 설명하고 있으며, 머신 러닝의 기초를 이해하는 데 중요한 요소들을 포함하고 있다. 각 단계에서의 데이터 활용과 모델 평가 방법은 머신 러닝 프로젝트의 성공을 좌우하는 핵심 요소임을 강조하고 있다.

## 2. 강의 내용 퀴즈

### 객관식 퀴즈

Q1. 선형 회귀 모델에서 훈련 데이터와 테스트 데이터를 분리하는 주된

In [ ]:
# Markdown 파일로 저장
with open("lecture_report.md", "w", encoding="utf-8") as f:
    f.write(result)

print("Markdown 파일로 저장 완료: lecture_report.md")

Markdown 파일로 저장 완료: lecture_report.md
